# "Smooth" statistics & smooth line plot

 Statistics "smooth" are calculated on the sample of two continuous variables (i.e., sample of points or line); It interpolates data points to create a smoother curve.

This notebook uses definitions from [DataFrame](https://kotlin.github.io/dataframe/overview.html).

## Usage

"Smooth" statistic is useful in case of the presence of overplotting or noises, making it easier to discern underlying trends and patterns. It can also be used to make a more pretty line with a small number of points.

## Arguments

* Input (mandatory):
  - `x` - numeric sample of input points `x` coordinates;
  - `y` - numeric sample of input points `y` coordinates;
* Parameters (optional):
  - `method: SmoothMethod` - smoothing model:
    - `SmoothMethod.Linear(confidenceLevel: Double)` -  linear model;
    - `SmoothMethod.Polynomial(degree: Int, confidenceLevel: Double)` -  polynomial model;
    - `SmoothMethod.LOESS(span: Double, loessCriticalSize: Int, samplingSeed: Long, confidenceLevel: Double)` -  Local Polynomial Regression model;
  - `smootherPointCount: Int` - number of sampled points;

### Generalized signature

The specific signature depends on the function, but all functions related to "smooth" statistic (which will be discussed further below - different variations of `statSmooth()`, `smoothLine()`) have approximately the same signature with the arguments above:

```
statSmoothArgs := 
   x, 
   y,
   method: SmoothMethod = SmoothMethod.LOESS(),
   smootherPointCount: Int = 100
```

The possible types of `x` and `y` depend on where a certain function is used. They can be simply `Iterable` (`List`, `Set`, etc.) or a reference to a column in a `DataFrame` (`String`, `ColumnAccessor`) or the `DataColumn` itself.

## Output statistics

| name      | type   | description                                         |
|-----------|--------|-----------------------------------------------------|
| Stat.x    | Double | `x` coordinate                                      |
| Stat.y    | Double | `y` coordinate                                      |
| Stat.yMin | Double | Lower pointwise confidence interval around the mean |
| Stat.yMax | Double | Upper pointwise confidence interval around the mean |
| Stat.se   | Double | Standard error                                      |

## StatSmooth plots

In [1]:
%use dataframe(0.11.1)
%use kandy(0.4.5-dev-29)
@file:Repository("https://packages.jetbrains.team/maven/p/kds/kotlin-ds-maven")
@file:DependsOn("org.jetbrains.kotlinx:kotlin-statistics-jvm:0.0.0-dev-11")

In [2]:
// to generate the data we use a standard java math library
// https://commons.apache.org/proper/commons-math/
import org.apache.commons.math3.distribution.NormalDistribution
import org.apache.commons.math3.distribution.UniformRealDistribution
import kotlin.random.Random

// generate line with formula
val xs = (-100..100).map { it / 50.0 }
val lineFormula = {x: Double -> 2.0 / (x * x + 0.5)}
// generate noises from normal distribution
val noises = NormalDistribution(0.0, 0.1).sample(xs.size).toList()
val ys = xs.zip(noises).map { lineFormula(it.first) + it.second }
// and drop 2/3 points
val (newXs, newYs) = xs.zip(ys).shuffled(Random(17)).take(xs.size * 1 / 3).sortedBy { it.first }.unzip()

In [3]:
// gather them into the DataFrame
val df = dataFrameOf(
    "speed" to newXs,
    "efficiency" to newYs
)

In [4]:
df.head(5)

<iframe onload="o_resize_iframe_out_1()" style="width:100%;" class="result_container" id="iframe_out_1" frameBorder="0" srcdoc=" <html>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}




:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 
<table class="dataframe" id="df_1660944384"></table>

<p class="dataframe_description">DataFrame: rowsCount = 5, columnsCount = 2</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for (let i = 0; i

`df` has a signature

<table>
  <thead>
    <tr>
      <th>speed</th>
      <th>efficiency</th>
    </tr>
  </thead>
</table>

Let's take a look at `StatSmooth` output DataFrame:

In [5]:
df.statSmooth("speed", "efficiency").head(5)

<iframe onload="o_resize_iframe_out_2()" style="width:100%;" class="result_container" id="iframe_out_2" frameBorder="0" srcdoc=" <html>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}




:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 
<table class="dataframe" id="df_1660944385"></table>

<p class="dataframe_description">DataFrame: rowsCount = 5, columnsCount = 1</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for (let i = 0; i

It has the following signature:

<table>
  <thead>
    <tr>
      <th alignt="left" colspan="5">Stat</th>
    </tr>
  </thead>
  <thead>
    <tr>
      <th>x</th>
      <th>y</th>
      <th>yMin</th>
      <th>yMax</th>
      <th>se</th>
    </tr>
  </thead>
</table>

As you can see, we got a `DataFrame` with one `ColumnGroup` called `Stat` which contains several columns with statics. For `statSmooth`, each row corresponds to one of line points. `Stat.x` is the column with this point `x` coordinate; `Stat.y` is points `y` coordinate; `Stat.yMin` - lower point of confidence level; `Stat.yMax` - upper point of confidence level; `Stat.se` - standard error.
`DataFrame` with "smooth" statistics is called `StatSmoothFrame`

### `statSmooth` transform

`statSmooth(statSmoothArgs) { /*new plotting context*/ }` modifies a plotting context - instead of original data (no matter was it empty or not) new `StatSmooth` dataset (calculated on given arguments; inputs can be provided as `Iterable` or as dataset column reference - by name as a `String`, as a `ColumnReference` or as a `DataColumn`) is used inside a new context (original dataset and primary context are not affected - you can add layers using initial dataset outside the `statSmooth` context). Since the old dataset is irrelevant, we cannot use references for its columns. But we can refer to the new ones. They are all contained in the `Stat` group and can be called inside the new context:

In [6]:
plot {
    statSmooth(newXs, newYs) {
        // new `StatSmooth` dataset here
        area {
            // use `Stat.*` columns for mappings
            x(Stat.x)
            y(Stat.y)
        }
    }
    points {
        x(newXs)
        y(newYs)
    }
} 

-2.0
 
 
 
 
 
 
 
 
 -1.5
 
 
 
 
 
 
 
 
 -1.0
 
 
 
 
 
 
 
 
 -0.5
 
 
 
 
 
 
 
 
 0.0
 
 
 
 
 
 
 
 
 0.5
 
 
 
 
 
 
 
 
 1.0
 
 
 
 
 
 
 
 
 1.5
 
 
 
 
 
 
 
 
 2.0
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0
 
 
 
 
 
 
 0.5
 
 
 
 
 
 
 1.0
 
 
 
 
 
 
 1.5
 
 
 
 
 
 
 2.0
 
 
 
 
 
 
 2.5
 
 
 
 
 
 
 3.0
 
 
 
 
 
 
 3.5
 
 
 
 
 
 
 4.0
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M24.54387721725996 325.03542680979035 L24.54387721725996 325.03542680979035 L29.50223625104985 321.185904324159 L34.46059528483974 317.3154716544548 L39.4189543186296 313.40787839406215 L44.37731335241949 309.4644282046945 L49.33567238620938 305.4891967668568 L54.29403141999927 301.4815075337274 L59.25239045378916 297.44099530982317 L64.21074948757905 293.36849197024947 L69.16910852136894 289.264794509359 L74.1274675551588 285.13068782954355 L79.08582658894869 280.96747354482625 L84.04418562273858 276.776485630395 L89.0025446565285 272.5583301223609 L93.96090369031839 268.31461023851404 L98.91926272410825 264.04971058990895 L103.87762175789817 259.7684847930367 L108.83598079168803 255.47578646438842 L113.79433982547792 251.17646922045517 L118.75269885926781 246.87570383476498 L123.7110578930577 242.58278597579903 L128.6694169268476 238.31613474874413 L133.62777596063745 234.086014314157 L138.58613499442737 229.8758148809064 L143.54449402821723 225.66496138791774 L148.50285306200712 221.40306598577092 L153.461212095797 216.7474855108323 L158.4195711295869 210.80855030112258 L163.3779301633768 204.38663946689383 L168.33628919716665 199.11480985499398 L173.29464823095654 193.32255123516603 L178.25300726474643 186.00931102598184 L183.21136629853635 178.1462931910754 L188.1697253323262 170.36143327738455 L193.12808436611613 162.18291186738543 L198.086443399906 154.1174224940655 L203.04480243369585 146.4935987411645 L208.00316146748577 138.4987113498863 L212.96152050127563 129.4062572642204 L217.91987953506555 120.86066241711674 L222.8782385688554 113.58757323366768 L227.83659760264533 109.16018591558355 L232.7949566364352 110.45884275116282 L237.75331567022508 114.64112513519825 L242.71167470401497 118.31728543300221 L247.67003373780486 120.03898697157129 L252.62839277159475 120.62861866874437 L257.5867518053846 121.03788411476933 L262.54511083917447 122.22981334626789 L267.5034698729644 125.17968512168079 L272.46182890675425 128.1322539879532 L277.4201879405442 124.77338406049537 L282.3785469743341 124.20778777169866 L287.33690600812395 126.34176872352623 L292.29526504191386 126.74996869834428 L297.2536240757037 130.81803723734217 L302.21198310949364 134.15752512074465 L307.1703421432835 139.450738174334 L312.12870117707337 146.09983974995907 L317.0870602108633 149.35071334074868 L322.04541924465315 145.1358554265862 L327.00377827844306 140.74506707226416 L331.9621373122329 141.70811564372886 L336.92049634602284 148.7855210691911 L341.8788553798127 156.92158313255712 L346.83721441360257 164.00231964406527 L351.7955734473925 171.16079668158775 L356.75393248118235 178.38953159417792 L361.71229151497226 185.38600456287932 L366.6706505487621 192.17113817474169 L371.62900958255204 198.77135130123762 L376.5873686163419 205.2130628138393 L381.54572765013177 211.56502239080146 L386.5040866839217 218.29488487832725 L391.46244571771155 225.08210887239323 L396.42080475150146 231.1902836134519 L401.3791637852913 236.69577995355257 L406.3375228190812 241.75370558314387 L411.2958818528711 246.89327439038368 L416.25424088666097 252.01966882126393 L421.2125999204509 256.6598657888229 L426.17095895424075 261.24442861880254 L431.12931798803066 265.7618372316204 L436.0876770218205 270.2093339418622 L441.0460360556104 274.60390416711135 L446.0043950894003 278.9468775163926 L450.96275412319017 283.2391634677408 L455.9211131569801 287.4844688487669 L460.87947219077 291.6848062490457 L465.83783122455986 295.8413325023954 L470.7961902583497 299.9548901003929 L475.7545492921396 304.02594745436096 L480.71290832592956 308.05380822716546 L485.6712673597194 312.03

In [7]:
df.plot { 
    statSmooth(speed, efficiency, method = SmoothMethod.Polynomial(2), smootherPointCount = 250) {
        ribbon { 
            x(Stat.x)
            yMin(Stat.yMin)
            yMax(Stat.yMax)
         }
    }
    // dataset is noe changed here
    points {
        x(speed)
        y(efficiency)
    }
 }

-2.0
 
 
 
 
 
 
 
 
 -1.5
 
 
 
 
 
 
 
 
 -1.0
 
 
 
 
 
 
 
 
 -0.5
 
 
 
 
 
 
 
 
 0.0
 
 
 
 
 
 
 
 
 0.5
 
 
 
 
 
 
 
 
 1.0
 
 
 
 
 
 
 
 
 1.5
 
 
 
 
 
 
 
 
 2.0
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1
 
 
 
 
 
 
 0
 
 
 
 
 
 
 1
 
 
 
 
 
 
 2
 
 
 
 
 
 
 3
 
 
 
 
 
 
 4
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M24.77449041313926 276.47060109492713 L24.77449041313926 276.47060109492713 L26.764409321825156 273.32130592365945 L28.75432823051102 270.1985681796695 L30.744247139196915 267.1023836667542 L32.73416604788278 264.0327480923148 L34.7240849565687 260.9896570648614 L36.71400386525457 257.97310609145006 L38.70392277394046 254.98309057505338 L40.69384168262633 252.01960581186245 L42.68376059131222 249.0826469885178 L44.67367949999809 246.1722091792698 L46.66359840868398 243.28828734306532 L48.653517317369875 240.43087632056097 L50.64343622605574 237.5999708310599 L52.633355134741635 234.79556546937235 L54.6232740434275 232.01765470259795 L56.61319295211342 229.266232866829 L58.60311186079926 226.5412941637731 L60.59303076948518 223.84283265729442 L62.58294967817105 221.17084226987322 L64.57286858685694 218.52531677898145 L66.5627874955428 215.90624981337493 L68.5527064042287 213.31363484930083 L70.54262531291457 210.74746520662015 L72.53254422160046 208.2077340448449 L74.52246313028635 205.6944343590897 L76.51238203897222 203.20755897593767 L78.50230094765814 200.7471005492215 L80.49221985634398 198.3130515557192 L82.4821387650299 195.9054042907659 L84.47205767371577 193.5241508637826 L86.46197658240166 191.1692831937228 L88.45189549108753 188.84079300443892 L90.44181439977342 186.53867181997032 L92.43173330845929 184.26291095975563 L94.42165221714518 182.0135015337712 L96.4115711258311 179.7904344376003 L98.40149003451694 177.59370034743546 L100.39140894320286 175.42328971501905 L102.3813278518887 173.27919276252675 L104.37124676057462 171.1613994773988 L106.36116566926049 169.0698996071265 L108.35108457794638 167.00468265399923 L110.34100348663225 164.96573786982063 L112.33092239531814 162.95305425060207 L114.320841304004 160.96662053124288 L116.3107602126899 159.0064251802077 L118.3006791213758 157.07245639421154 L120.29059803006166 155.16470209292606 L122.28051693874758 153.2831499137195 L124.27043584743342 151.42778720644583 L126.26035475611934 149.59860102829816 L128.2502736648052 147.79557813874487 L130.2401925734911 146.01870499456584 L132.23011148217697 144.26796774501022 L134.22003039086286 142.54335222709682 L136.20994929954873 140.8448439610804 L138.19986820823462 139.17242814610958 L140.1897871169205 137.5260896561025 L142.17970602560638 135.9058130358695 L144.1696249342923 134.31158249751238 L146.15954384297814 132.7433819171345 L148.14946275166403 131.20119483189384 L150.13938166034993 129.68500443743758 L152.12930056903582 128.1947935857552 L154.11921947772169 126.7305447834919 L156.10913838640758 125.29224019076378 L158.09905729509347 123.87986162052047 L160.08897620377934 122.49339053850144 L162.0788951124652 121.1328080638348 L164.06881402115113 119.79809497032903 L166.058732929837 118.48923168851076 L168.0486518385229 117.20619830846147 L170.03857074720878 115.94897458351028 L172.02848965589465 114.71753993483856 L174.0184085645805 113.5118734570554 L176.0083274732664 112.33195392480266 L177.9982463819523 111.17775980044908 L179.98816529063816 110.04926924293392 L181.97808419932406 108.9464601178193 L183.96800310800995 107.86931000861102 L185.95792201669585 106.81779622940553 L187.94784092538174 105.79189583892037 L189.9377598340676 104.79158565596188 L191.92767874275347 103.81684227638274 L193.91759765143937 102.86764209157693 L195.90751656012526 101.94396130855722 L197.89743546881112 101.04577597165371 L199.887354377497 100.173061985868 L201.8772732861829 99.3257951419092 L203.86719219486878 98.50395114293264 L205.8571111035547 97.7075056329922 L207.84703001224057 96.93643422720899 L209.83694892092643 96.19071254364903 L211.82686782961233 95.47031623689139 L213.81678673829822 94.7752

### `smoothLine` layer

`smoothLine` layer is a shortcut for fast plotting a smoothed line:

In [8]:
val smoothLineLayerPlot = plot {
    smoothLine(newXs, newYs)
    layout.title = "`smoothLine()` layer"
}
smoothLineLayerPlot

-2.0
 
 
 
 
 
 
 
 
 -1.5
 
 
 
 
 
 
 
 
 -1.0
 
 
 
 
 
 
 
 
 -0.5
 
 
 
 
 
 
 
 
 0.0
 
 
 
 
 
 
 
 
 0.5
 
 
 
 
 
 
 
 
 1.0
 
 
 
 
 
 
 
 
 1.5
 
 
 
 
 
 
 
 
 2.0
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0
 
 
 
 
 
 
 0.5
 
 
 
 
 
 
 1.0
 
 
 
 
 
 
 1.5
 
 
 
 
 
 
 2.0
 
 
 
 
 
 
 2.5
 
 
 
 
 
 
 3.0
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M24.54387721725996 292.54075652967344 L24.54387721725996 292.54075652967344 L29.50223625104985 287.58349898844125 L34.46059528483974 282.5993141677684 L39.4189543186296 277.56727545895365 L44.37731335241949 272.48906166420807 L49.33567238620938 267.3699212781162 L54.29403141999927 262.20898307136406 L59.25239045378916 257.00577676015195 L64.21074948757905 251.76137359920142 L69.16910852136894 246.47679992522836 L74.1274675551588 241.15306650341603 L79.08582658894869 235.79184949904374 L84.04418562273858 230.39486675077427 L89.0025446565285 224.96289868711392 L93.96090369031839 219.49800986633156 L98.91926272410825 214.0058466111327 L103.87762175789817 208.49265921023303 L108.83598079168803 202.96469795234822 L113.79433982547792 197.4282131261939 L118.75269885926781 191.8898634423404 L123.7110578930577 186.3616194822953 L128.6694169268476 180.86720061604032 L133.62777596063745 175.4198246095489 L138.58613499442737 169.9981020534459 L143.54449402821723 164.5755372259752 L148.50285306200712 159.08724271643678 L153.461212095797 153.09197667906543 L158.4195711295869 145.44405892689923 L163.3779301633768 137.17418492799982 L168.33628919716665 130.38533846789252 L173.29464823095654 122.92630482642514 L178.25300726474643 113.50861333010565 L183.21136629853635 103.38294070332512 L188.1697253323262 93.3579166493999 L193.12808436611613 82.82595142847251 L198.086443399906 72.43954423930631 L203.04480243369585 62.621895963926676 L208.00316146748577 52.32640711832346 L212.96152050127563 40.61751679420769 L217.91987953506555 29.61284942098365 L222.8782385688554 20.24686277269575 L227.83659760264533 14.545454545454561 L232.7949566364352 16.21781167953486 L237.75331567022508 21.603583801975617 L242.71167470401497 26.337592611402897 L247.67003373780486 28.55472940374011 L252.62839277159475 29.31403297665622 L257.5867518053846 29.84106827161702 L262.54511083917447 31.375985964270114 L267.5034698729644 35.174710133025144 L272.46182890675425 38.97690750502869 L277.4201879405442 34.651492251563184 L282.3785469743341 33.92314049406707 L287.33690600812395 36.671193845042126 L292.29526504191386 37.196857069779924 L297.2536240757037 42.43554927601326 L302.21198310949364 46.7360051262238 L307.1703421432835 53.55238830756559 L312.12870117707337 62.114829246985266 L317.0870602108633 66.30117123711173 L322.04541924465315 60.87344967990737 L327.00377827844306 55.2191721006331 L331.9621373122329 56.45934664469297 L336.92049634602284 65.5733397564845 L341.8788553798127 76.05062756818035 L346.83721441360257 85.16891031627947 L351.7955734473925 94.38730413277281 L356.75393248118235 103.69617316594824 L361.71229151497226 112.70594477153844 L366.6706505487621 121.4435622010302 L371.62900958255204 129.94304659609165 L376.5873686163419 138.23841909839038 L381.54572765013177 146.41821273046565 L386.5040866839217 155.08465426763502 L391.46244571771155 163.8249636057938 L396.42080475150146 171.69082110144396 L401.3791637852913 178.78057401289226 L406.3375228190812 185.29396367716143 L411.2958818528711 191.9124900849096 L416.25424088666097 198.51405106985447 L421.2125999204509 204.48950685588875 L426.17095895424075 210.39331927600034 L431.12931798803066 216.21065324308483 L436.0876770218205 221.9379575227053 L441.0460360556104 227.59710524061194 L446.0043950894003 233.18980861290152 L450.96275412319017 238.7172388281731 L455.9211131569801 244.18416938734697 L460.87947219077 249.5931920209262 L465.83783122455986 254.94579645216592 L470.7961902583497 260.2430676072458 L475.7545492921396 265.4856086871005 L480.71290832592956 270.67252297726185 L485.6712673597194 275.8053116969617 L490.6296263935093 280.8855475551918 L

In [9]:
// compare it with `statSmooth` + usual `line`
val statSmoothAndLinePlot = plot {
    statSmooth(newXs, newYs) {
        line {
            x(Stat.x)
            y(Stat.y)
        }
    }
   layout.title = "`statSmooth()` + non-statistical `line` layer"
} 
plotGrid(listOf(smoothLineLayerPlot, statSmoothAndLinePlot))

-2
 
 
 
 
 
 
 
 
 -1
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 
 
 1
 
 
 
 
 
 
 
 
 2
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0
 
 
 
 
 
 
 0.5
 
 
 
 
 
 
 1.0
 
 
 
 
 
 
 1.5
 
 
 
 
 
 
 2.0
 
 
 
 
 
 
 2.5
 
 
 
 
 
 
 3.0
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M17.634786308169026 292.54075652967344 L17.634786308169026 292.54075652967344 L21.197369400728434 287.58349898844125 L24.75995249328784 282.5993141677684 L28.32253558584722 277.56727545895365 L31.885118678406627 272.48906166420807 L35.447701770966006 267.3699212781162 L39.01028486352541 262.20898307136406 L42.57286795608482 257.00577676015195 L46.13545104864423 251.76137359920142 L49.698034141203635 246.47679992522836 L53.260617233763014 241.15306650341603 L56.82320032632242 235.79184949904374 L60.3857834188818 230.39486675077427 L63.94836651144121 224.96289868711392 L67.51094960400061 219.49800986633156 L71.07353269656 214.0058466111327 L74.63611578911942 208.49265921023303 L78.19869888167881 202.96469795234822 L81.7612819742382 197.4282131261939 L85.32386506679761 191.8898634423404 L88.886448159357 186.3616194822953 L92.44903125191641 180.86720061604032 L96.01161434447579 175.4198246095489 L99.57419743703521 169.9981020534459 L103.1367805295946 164.5755372259752 L106.699363622154 159.08724271643678 L110.2619467147134 153.09197667906543 L113.82452980727281 145.44405892689923 L117.3871128998322 137.17418492799982 L120.94969599239158 130.38533846789252 L124.51227908495099 122.92630482642514 L128.0748621775104 113.50861333010565 L131.6374452700698 103.38294070332512 L135.2000283626292 93.3579166493999 L138.7626114551886 82.82595142847251 L142.325194547748 72.43954423930631 L145.88777764030738 62.621895963926676 L149.45036073286678 52.32640711832346 L153.0129438254262 40.61751679420769 L156.5755269179856 29.61284942098365 L160.138110010545 20.24686277269575 L163.70069310310438 14.545454545454561 L167.26327619566376 16.21781167953486 L170.82585928822317 21.603583801975617 L174.38844238078258 26.337592611402897 L177.95102547334199 28.55472940374011 L181.5136085659014 29.31403297665622 L185.0761916584608 29.84106827161702 L188.63877475102018 31.375985964270114 L192.20135784357956 35.174710133025144 L195.76394093613897 38.97690750502869 L199.3265240286984 34.651492251563184 L202.88910712125778 33.92314049406707 L206.45169021381716 36.671193845042126 L210.0142733063766 37.196857069779924 L213.57685639893597 42.43554927601326 L217.13943949149538 46.7360051262238 L220.70202258405476 53.55238830756559 L224.26460567661414 62.114829246985266 L227.82718876917357 66.30117123711173 L231.38977186173295 60.87344967990737 L234.9523549542924 55.2191721006331 L238.51493804685177 56.45934664469297 L242.07752113941117 65.5733397564845 L245.64010423197055 76.05062756818035 L249.20268732452996 85.16891031627947 L252.76527041708937 94.38730413277281 L256.3278535096488 103.69617316594824 L259.8904366022082 112.70594477153844 L263.45301969476753 121.4435622010302 L267.015602787327 129.94304659609165 L270.57818587988635 138.23841909839038 L274.14076897244576 146.41821273046565 L277.70335206500516 155.08465426763502 L281.2659351575645 163.8249636057938 L284.828518250124 171.69082110144396 L288.3911013426833 178.78057401289226 L291.95368443524274 185.29396367716143 L295.51626752780214 191.9124900849096 L299.07885062036155 198.51405106985447 L302.64143371292096 204.48950685588875 L306.20401680548036 210.39331927600034 L309.7665998980398 216.21065324308483 L313.3291829905991 221.9379575227053 L316.89176608315853 227.59710524061194 L320.45434917571794 233.18980861290152 L324.0169322682773 238.7172388281731 L327.57951536083675 244.18416938734697 L331.1420984533961 249.5931920209262 L334.70468154595557 254.94579645216592 L338.2672646385149 260.2430676072458 L341.8298477310743 265.4856086871005 L345.39243082363373 270.67252297726185 L348.95501391619314 275.8053116969617 L352.51759700875255 280.8855475551918 L356.08018010131195 285.9105142994201 L359.6427631938713 290.87621271074613 L363.20534628643077 2

`smoothLine` uses `statSmooth` and `line` and performs coordinates mappings under the hood. And of course we can customize `smoothLine` layer: `smoothLine()` optionally opens a new context, where we can configure line (as in usual context opened by `line { ... }`) - even change coordinate mappings from default ones. `StatSmooth` dataset of `smoothLine` is also can be accessed here.

In [10]:
df.plot {
    smoothLine(speed, efficiency, SmoothMethod.LOESS(span = 0.1), speed.size()) {
         // change a column mapped on `y` to `Stat.scaled`
        y(Stat.yMax)
        color = Color.RED
        width = 4.0
    }
    points {
        x(speed)
        y(efficiency)
    }
}

-2.0
 
 
 
 
 
 
 
 
 -1.5
 
 
 
 
 
 
 
 
 -1.0
 
 
 
 
 
 
 
 
 -0.5
 
 
 
 
 
 
 
 
 0.0
 
 
 
 
 
 
 
 
 0.5
 
 
 
 
 
 
 
 
 1.0
 
 
 
 
 
 
 
 
 1.5
 
 
 
 
 
 
 
 
 2.0
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.5
 
 
 
 
 
 
 1.0
 
 
 
 
 
 
 1.5
 
 
 
 
 
 
 2.0
 
 
 
 
 
 
 2.5
 
 
 
 
 
 
 3.0
 
 
 
 
 
 
 3.5
 
 
 
 
 
 
 4.0
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M24.54387721725996 312.3611712794767 L24.54387721725996 312.3611712794767 L31.98141576794478 312.45613907108253 L39.4189543186296 314.3561825214283 L46.85649286931445 313.69262816744146 L54.29403141999927 308.84359942163553 L61.73156997068409 304.05021854737726 L69.16910852136894 299.45770869858626 L76.60664707205376 297.2669823901058 L84.04418562273858 295.6322189396905 L91.48172417342343 294.37087042672835 L98.91926272410825 293.31833196316256 L106.3568012747931 290.5485846433922 L113.79433982547792 284.02697625862345 L121.23187837616274 273.42549585196184 L128.6694169268476 263.32707932115153 L136.1069554775324 250.14871443592244 L143.54449402821723 248.03656648589453 L150.98203257890205 244.18930483688132 L158.41957112958687 242.12538272200695 L165.85710968027172 234.02180238332613 L173.29464823095654 216.99196967715312 L180.73218678164136 189.63737673828052 L188.1697253323262 166.62043488561267 L195.60726388301106 157.96673211081324 L203.04480243369585 146.9053327201345 L210.4823409843807 136.59041699561507 L217.9198795350655 122.55253624775978 L225.35741808575034 99.73114286006779 L232.7949566364352 83.2267752201642 L240.23249518712 71.73464449604688 L247.67003373780486 58.64623747535626 L255.10757228848968 44.394975617491525 L262.54511083917447 30.923286135253477 L269.9826493898593 18.683666349086877 L277.42018794054417 26.91623291800039 L284.85772649122896 42.14054914301249 L292.2952650419138 55.94097764015072 L299.73280359259866 64.3153412958311 L307.1703421432835 78.53297165464329 L314.6078806939683 91.64105940323452 L322.04541924465315 112.06491506283098 L329.48295779533794 132.17459608807187 L336.9204963460228 161.441299615479 L344.3580348967076 180.16266053407955 L351.7955734473925 197.41531119547926 L359.2331119980773 212.02646178492262 L366.6706505487621 221.2308125714273 L374.1081890994469 230.2019926679116 L381.54572765013177 243.51578766845742 L388.9832662008166 251.9650348538772 L396.4208047515014 262.3386739775467 L403.8583433021862 263.689634866822 L411.29588185287105 262.55694965905377 L418.7334204035559 279.66094657609653 L426.17095895424075 286.55855759102997 L433.6084975049256 290.7546000589533 L441.0460360556104 293.97255711835095 L448.4835746062952 291.62745270670064 L455.92111315698 301.25128475191684 L463.3586517076649 312.03425688007536 L470.7961902583497 313.99093201675294 L478.2337288090346 313.0128634181098 L485.6712673597194 312.37184411459054 L493.1088059104042 313.3152000274436 L500.546344461089 315.85955373393574 L507.98388301177386 319.09758856679923 L515.4214215624587 322.5408636184799 " fill="none" stroke-width="8.8" stroke="rgb(238,102,102)" stroke-opacity="1.0">
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 efficiency
 
 
 
 
 speed

## Grouped `statSmoth`

`statSmooth` can be applied for grouped data - statistics will be calculated on each group independently but with equal categories. This application returns a new `GroupBy` dataset with the same keys as the old one but with `StatSmooth` groups instead of old ones.

In [11]:
// generate two lines
// lines formulas
val fA = { x: Double -> 0.02 * x * x * x - 0.2 * x * x + 0.1 * x + 2.1 }
val fB = { x: Double -> -0.1 * x * x * x + 0.5 * x * x - 0.8 }
val xRange = (-500..500).map { it / 100.0 }
val noisesA = NormalDistribution(0.0, 0.05).sample(xRange.size).toList()
val noisesB = NormalDistribution(0.0, 0.2).sample(xRange.size).toList()
val valuesA = xRange.zip(noisesA).map { fA(it.first) + it.second }
val valuesB = xRange.zip(noisesB).map { fB(it.first) + it.second }

val (xsA, ysA) = xRange.zip(valuesA).shuffled(Random(17)).take(xRange.size * 1 / 3).sortedBy { it.first }.unzip()
val (xsB, ysB) = xRange.zip(valuesB).shuffled(Random(17)).take(xRange.size * 1 / 3).sortedBy { it.first }.unzip()

In [23]:
// gather them into `DataFrame` with "A" and "B" keys in "category" column
val valuesDF = dataFrameOf(
    "time" to xsA + xsB,
    "value" to ysA + ysB,
    "category" to List(xsA.size) { "A" } + List(xsB.size) { "B" }
)
valuesDF.head(5)

<iframe onload="o_resize_iframe_out_3()" style="width:100%;" class="result_container" id="iframe_out_3" frameBorder="0" srcdoc=" <html>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}




:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 
<table class="dataframe" id="df_1660944386"></table>

<p class="dataframe_description">DataFrame: rowsCount = 5, columnsCount = 3</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for (let i = 0; i

It has the following signature:

<table>
  <thead>
    <tr>
      <th>time</th>
      <th>value</th>
      <th>category</th>
    </tr>
  </thead>
</table>

In [24]:
// group it by "category"
val groupedDF = valuesDF.groupBy { category }
groupedDF

<iframe onload="o_resize_iframe_out_4()" style="width:100%;" class="result_container" id="iframe_out_4" frameBorder="0" srcdoc=" <html>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}




:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 
<table class="dataframe" id="df_1660944387"></table>

<p class="dataframe_description">GroupBy</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for (let i = 0; i < df.cols.length; i++) {
 if (df.c

Now we have a `GroupBy` with a signature

<table>
  <thead>
    <tr>
      <th>key: [category]</th>
      <th>group: DataFrame[time|value|category]</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>A</td>
      <td>A-Group</td>
    </tr>
    <tr>
      <td>B</td>
      <td>B-Group</td>
    </tr>
  </tbody>
</table>

In [25]:
groupedDF.statSmooth { x(time); y(value) }

<iframe onload="o_resize_iframe_out_5()" style="width:100%;" class="result_container" id="iframe_out_5" frameBorder="0" srcdoc=" <html>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}




:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 
<table class="dataframe" id="df_1660944390"></table>

<p class="dataframe_description">GroupBy</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for (let i = 0; i < df.cols.length; i++) {
 if (df.c

After `statSmooth` applying it's still a `GroupBy` but with different signature of `group` - all groups have the same signature as usual `DataFrame` after `statSmooth` applying (i.e. `StatSmoothFrame`):

<table>
  <thead>
    <tr>
      <th>key: [drv]</th>
      <th>group: StaSmoothFrame</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>"A"</td>
      <td>"A"-Group</td>
    </tr>
    <tr>
      <td>"B"</td>
      <td>"B"-Group</td>
    </tr>
  </tbody>
</table>

As you can see, we did indeed do a `statSmooth` transformation within groups, the grouping keys did not change.

The plotting process doesn't change much - we do everything the same. 

In [29]:
groupedDF.plot {
    statSmooth(time, value) {
        line {
            x(Stat.x)
            y(Stat.y)
        }
    }
}

-4
 
 
 
 
 
 
 
 
 -2
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 
 
 2
 
 
 
 
 
 
 
 
 4
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -5
 
 
 
 
 
 
 0
 
 
 
 
 
 
 5
 
 
 
 
 
 
 10
 
 
 
 
 
 
 15
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M24.77449041313926 328.3636363636364 L24.77449041313926 328.3636363636364 L29.77943797134921 325.800102733952 L34.78438552955916 323.24504427878355 L39.78933308776911 320.69908436517136 L44.79428064597906 318.1628894948984 L49.799228204189006 315.637193218421 L54.804175762398955 313.12278249507153 L59.809123320608904 310.62046752749603 L64.81407087881885 308.1310180161044 L69.8190184370288 305.65506093754885 L74.82396599523875 303.19296204437603 L79.8289135534487 300.74480429967554 L84.83386111165868 298.3104528634424 L89.83880866986863 295.88958305977184 L94.84375622807858 293.4817286487229 L99.84870378628852 291.08631329461855 L104.85365134449847 288.7027022933154 L109.85859890270842 286.3301987065473 L114.86354646091837 283.9679444205358 L119.86849401912832 281.61492795797903 L124.87344157733827 279.26992782449497 L129.87838913554822 276.93145040974144 L134.88333669375817 274.5975815557226 L139.88828425196812 272.2658912965302 L144.89323181017807 269.92807143377274 L149.898179368388 267.6983569489861 L154.90312692659796 265.6076993361114 L159.9080744848079 263.49440119084034 L164.91302204301786 261.6808120590238 L169.9179696012278 259.99746731374614 L174.92291715943776 258.04995285697566 L179.9278647176477 256.27511114804565 L184.93281227585766 254.6525116869527 L189.9377598340676 252.99506745922716 L194.94270739227755 251.66045567999586 L199.9476549504875 250.59053832816574 L204.95260250869745 249.43763561018292 L209.9575500669074 248.41935425487057 L214.96249762511735 247.44231028943207 L219.9674451833273 246.4311806502593 L224.97239274153725 245.63095527669523 L229.9773402997472 244.77075274517105 L234.98228785795715 243.71661857309266 L239.9872354161671 243.1271694436694 L244.99218297437704 242.59109401426707 L249.997130532587 242.0483040037158 L255.00207809079694 241.7666393094316 L260.0070256490069 241.42312590285925 L265.0119732072169 241.06493423139753 L270.01692076542685 241.0278513100271 L275.0218683236368 240.77961786863844 L280.02681588184674 240.70746159036042 L285.0317634400567 240.780839622281 L290.03671099826664 240.80568871699873 L295.0416585564766 240.86070850898682 L300.04660611468654 240.8885121065922 L305.0515536728965 240.9125123710619 L310.05650123110644 240.82324231502423 L315.0614487893164 241.1483681166198 L320.06639634752634 241.45785577787657 L325.0713439057363 241.76274349693324 L330.07629146394623 242.15224639970785 L335.0812390221562 242.41211354731954 L340.08618658036613 242.8001234784711 L345.0911341385761 243.13454014507303 L350.09608169678603 243.5536597123711 L355.101029254996 243.95352987419673 L360.1059768132059 244.4750003743899 L365.1109243714159 245.01554172906577 L370.1158719296258 245.54600985510592 L375.1208194878358 246.06958270105923 L380.1257670460457 246.62325764819394 L385.1307146042557 247.2842579698816 L390.1356621624656 247.8959592495634 L395.14060972067557 248.58381919489784 L400.1455572788855 249.29742788610986 L405.15050483709547 249.99042926889362 L410.1554523953054 250.67032302418195 L415.16039995351537 251.41954539998363 L420.1653475117253 252.16811331979432 L425.17029506993526 252.9165670649028 L430.17524262814527 253.66538051105314 L435.18019018635516 254.41484450105492 L440.18513774456517 255.16512611514048 L445.19008530277506 255.91631440384094 L450.19503286098507 256.6684594192351 L455.19998041919496 257.42157423069676 L460.20492797740496 258.1756487633006 L465.20987553561486 258.9306564392321 L470.21482309382486 259.6865520157213 L475.21977065203475 260.4432778593571 L480.22471821024476 261.2007693283015 L485.22966576845465 261.9589568502778 L490.23461332666466 262.7177678707485 L495.23956088487455 263.4771162258311 L500.24450844308456 264.23690853202197 L505.24945600129456 264.99705372797007 L510.25440355950445 265.7574558306045 L515.2593511177145 

As you can see, there are two lines because we have two groups of data. To distinguish them, we need to add mapping to the color from the key. This is easy - key is available in the context

In [33]:
groupedDF.plot {
    statSmooth(time, value, method = SmoothMethod.Polynomial(3)) {
        line {
            x(Stat.x)
            y(Stat.y)
            color(key.category)
        }
    }
}

-4
 
 
 
 
 
 
 
 
 -2
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 
 
 2
 
 
 
 
 
 
 
 
 4
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -5
 
 
 
 
 
 
 0
 
 
 
 
 
 
 5
 
 
 
 
 
 
 10
 
 
 
 
 
 
 15
 
 
 
 
 
 
 20
 
 
 
 
 
 
 25
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M20.82183586673196 328.3636363636364 L20.82183586673196 328.3636363636364 L25.028267354960633 324.6221538794846 L29.234698843189307 320.9857856388112 L33.44113033141798 317.4532462678145 L37.64756181964668 314.02325039269255 L41.853993307875356 310.69451263964345 L46.06042479610403 307.4657476348654 L50.266856284332704 304.33567000455656 L54.47328777256138 301.302994374915 L58.67971926079005 298.3664353721388 L62.886150749018725 295.52470762242626 L67.0925822372474 292.77652575197544 L71.2990137254761 290.1206043869844 L75.50544521370477 287.5556581536514 L79.71187670193345 285.0804016781745 L83.91830819016212 282.69354958675183 L88.1247396783908 280.3938165055816 L92.33117116661947 278.1799170608619 L96.53760265484814 276.05056587879085 L100.74403414307683 274.00447758556663 L104.9504656313055 272.04036680738733 L109.15689711953418 270.1569481704511 L113.36332860776285 268.35293630095606 L117.56976009599154 266.6270458251004 L121.77619158422023 264.97799136908225 L125.9826230724489 263.4044875590997 L130.18905456067756 261.9052490213509 L134.39548604890626 260.47899038203406 L138.60191753713494 259.1244262673472 L142.8083490253636 257.8402713034885 L147.01478051359229 256.6252401166562 L151.22121200182096 255.47804733304827 L155.42764349004963 254.39740757886292 L159.6340749782783 253.3820354802983 L163.840506466507 252.43064566355253 L168.04693795473565 251.54195275482377 L172.25336944296436 250.7146713803101 L176.459800931193 249.94751616620974 L180.6662324194217 249.23920173872074 L184.87266390765038 248.5884427240413 L189.07909539587905 247.99395374836953 L193.28552688410772 247.4544494379036 L197.4919583723364 246.9686444188416 L201.69838986056507 246.53525331738166 L205.90482134879375 246.15299075972197 L210.11125283702242 245.82057137206064 L214.3176843252511 245.5367097805958 L218.52411581347977 245.30012061152559 L222.7305473017085 245.10951849104814 L226.93697878993717 244.96361804536159 L231.14341027816585 244.8611339006641 L235.34984176639452 244.8007806831538 L239.5562732546232 244.7812730190288 L243.76270474285187 244.80132553448723 L247.96913623108054 244.85965285572726 L252.1755677193092 244.954969608947 L256.3819992075379 245.08599042034464 L260.5884306957666 245.25142991611827 L264.79486218399524 245.45000272246602 L269.00129367222394 245.68042346558607 L273.2077251604526 245.9414067716765 L277.4141566486813 246.2316672669355 L281.62058813691 246.54991957756116 L285.82701962513863 246.89487832975163 L290.0334511133673 247.2652581497051 L294.239882601596 247.65977366361963 L298.4463140898247 248.0771394976934 L302.6527455780533 248.51607027812452 L306.85917706628203 248.97528063111113 L311.0656085545107 249.4534851828514 L315.2720400427394 249.94939855954345 L319.478471530968 250.4617353873854 L323.6849030191967 250.98921029257542 L327.8913345074254 251.53053790131162 L332.0977659956541 252.08443283979213 L336.3041974838827 252.6496097342151 L340.5106289721114 253.22478321077867 L344.7170604603401 253.80866789568097 L348.92349194856877 254.39997841512013 L353.12992343679747 254.99742939529432 L357.3363549250261 255.5997354624016 L361.5427864132548 256.2056112426402 L365.74921790148346 256.81377136220823 L369.9556493897122 257.42293044730377 L374.16208087794087 258.031803124125 L378.3685123661695 258.6391040188701 L382.57494385439816 259.2435477577372 L386.7813753426269 259.8438489669243 L390.98780683085556 260.43872227262966 L395.1942383190842 261.02688230105144 L399.40066980731285 261.6070436783877 L403.6071012955416 262.1779210308366 L407.81353278377026 262.7382289845963 L412.01996427199896 263.2866821658649 L416.2263957602276 263.82199520084055 L420.4328272484563 264.3428827157214 L424.639258736685 264.84805933670555 L428.84569022491365 265.3362396899912 

The `smoothLine()` layer also works. Moreover, if we have exactly one grouping key, a mapping from it to `color` will be created by default.

In [34]:
groupedDF.plot {
    smoothLine(time, value)
}

-4
 
 
 
 
 
 
 
 
 -2
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 
 
 2
 
 
 
 
 
 
 
 
 4
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -5
 
 
 
 
 
 
 0
 
 
 
 
 
 
 5
 
 
 
 
 
 
 10
 
 
 
 
 
 
 15
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M20.82183586673196 328.3636363636364 L20.82183586673196 328.3636363636364 L25.028267354960633 325.800102733952 L29.234698843189307 323.24504427878355 L33.44113033141798 320.69908436517136 L37.64756181964668 318.1628894948984 L41.853993307875356 315.637193218421 L46.06042479610403 313.12278249507153 L50.266856284332704 310.62046752749603 L54.47328777256138 308.1310180161044 L58.67971926079005 305.65506093754885 L62.886150749018725 303.19296204437603 L67.0925822372474 300.74480429967554 L71.2990137254761 298.3104528634424 L75.50544521370477 295.88958305977184 L79.71187670193345 293.4817286487229 L83.91830819016212 291.08631329461855 L88.1247396783908 288.7027022933154 L92.33117116661947 286.3301987065473 L96.53760265484814 283.9679444205358 L100.74403414307683 281.61492795797903 L104.9504656313055 279.26992782449497 L109.15689711953418 276.93145040974144 L113.36332860776285 274.5975815557226 L117.56976009599154 272.2658912965302 L121.77619158422023 269.92807143377274 L125.9826230724489 267.6983569489861 L130.18905456067756 265.6076993361114 L134.39548604890626 263.49440119084034 L138.60191753713494 261.6808120590238 L142.8083490253636 259.99746731374614 L147.01478051359229 258.04995285697566 L151.22121200182096 256.27511114804565 L155.42764349004963 254.6525116869527 L159.6340749782783 252.99506745922716 L163.840506466507 251.66045567999586 L168.04693795473565 250.59053832816574 L172.25336944296436 249.43763561018292 L176.459800931193 248.41935425487057 L180.6662324194217 247.44231028943207 L184.87266390765038 246.4311806502593 L189.07909539587905 245.63095527669523 L193.28552688410772 244.77075274517105 L197.4919583723364 243.71661857309266 L201.69838986056507 243.1271694436694 L205.90482134879375 242.59109401426707 L210.11125283702242 242.0483040037158 L214.3176843252511 241.7666393094316 L218.52411581347977 241.42312590285925 L222.7305473017085 241.06493423139753 L226.93697878993717 241.0278513100271 L231.14341027816585 240.77961786863844 L235.34984176639452 240.70746159036042 L239.5562732546232 240.780839622281 L243.76270474285187 240.80568871699873 L247.96913623108054 240.86070850898682 L252.1755677193092 240.8885121065922 L256.3819992075379 240.9125123710619 L260.5884306957666 240.82324231502423 L264.79486218399524 241.1483681166198 L269.00129367222394 241.45785577787657 L273.2077251604526 241.76274349693324 L277.4141566486813 242.15224639970785 L281.62058813691 242.41211354731954 L285.82701962513863 242.8001234784711 L290.0334511133673 243.13454014507303 L294.239882601596 243.5536597123711 L298.4463140898247 243.95352987419673 L302.6527455780533 244.4750003743899 L306.85917706628203 245.01554172906577 L311.0656085545107 245.54600985510592 L315.2720400427394 246.06958270105923 L319.478471530968 246.62325764819394 L323.6849030191967 247.2842579698816 L327.8913345074254 247.8959592495634 L332.0977659956541 248.58381919489784 L336.3041974838827 249.29742788610986 L340.5106289721114 249.99042926889362 L344.7170604603401 250.67032302418195 L348.92349194856877 251.41954539998363 L353.12992343679747 252.16811331979432 L357.3363549250261 252.9165670649028 L361.5427864132548 253.66538051105314 L365.74921790148346 254.41484450105492 L369.9556493897122 255.16512611514048 L374.16208087794087 255.91631440384094 L378.3685123661695 256.6684594192351 L382.57494385439816 257.42157423069676 L386.7813753426269 258.1756487633006 L390.98780683085556 258.9306564392321 L395.1942383190842 259.6865520157213 L399.40066980731285 260.4432778593571 L403.6071012955416 261.2007693283015 L407.81353278377026 261.9589568502778 L412.01996427199896 262.7177678707485 L416.2263957602276 263.4771162258311 L420.4328272484563 264.23690853202197 L424.639258736685 264.99705372797007 L428.84569022491365 265.7574558306045 L433.05212171314236 266.518025748

We can customize it like we used to. From the differences — access to `key` columns:

In [40]:
groupedDF.plot {
    smoothLine(time, value) {
        color = Color.GREEN
        type(key.category) {
            scale = categorical()
        }
    }
}//.toLetsPlot()

-4
 
 
 
 
 
 
 
 
 -2
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 
 
 2
 
 
 
 
 
 
 
 
 4
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -5
 
 
 
 
 
 
 0
 
 
 
 
 
 
 5
 
 
 
 
 
 
 10
 
 
 
 
 
 
 15
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M20.82183586673196 328.3636363636364 L20.82183586673196 328.3636363636364 L25.028267354960633 325.800102733952 L29.234698843189307 323.24504427878355 L33.44113033141798 320.69908436517136 L37.64756181964668 318.1628894948984 L41.853993307875356 315.637193218421 L46.06042479610403 313.12278249507153 L50.266856284332704 310.62046752749603 L54.47328777256138 308.1310180161044 L58.67971926079005 305.65506093754885 L62.886150749018725 303.19296204437603 L67.0925822372474 300.74480429967554 L71.2990137254761 298.3104528634424 L75.50544521370477 295.88958305977184 L79.71187670193345 293.4817286487229 L83.91830819016212 291.08631329461855 L88.1247396783908 288.7027022933154 L92.33117116661947 286.3301987065473 L96.53760265484814 283.9679444205358 L100.74403414307683 281.61492795797903 L104.9504656313055 279.26992782449497 L109.15689711953418 276.93145040974144 L113.36332860776285 274.5975815557226 L117.56976009599154 272.2658912965302 L121.77619158422023 269.92807143377274 L125.9826230724489 267.6983569489861 L130.18905456067756 265.6076993361114 L134.39548604890626 263.49440119084034 L138.60191753713494 261.6808120590238 L142.8083490253636 259.99746731374614 L147.01478051359229 258.04995285697566 L151.22121200182096 256.27511114804565 L155.42764349004963 254.6525116869527 L159.6340749782783 252.99506745922716 L163.840506466507 251.66045567999586 L168.04693795473565 250.59053832816574 L172.25336944296436 249.43763561018292 L176.459800931193 248.41935425487057 L180.6662324194217 247.44231028943207 L184.87266390765038 246.4311806502593 L189.07909539587905 245.63095527669523 L193.28552688410772 244.77075274517105 L197.4919583723364 243.71661857309266 L201.69838986056507 243.1271694436694 L205.90482134879375 242.59109401426707 L210.11125283702242 242.0483040037158 L214.3176843252511 241.7666393094316 L218.52411581347977 241.42312590285925 L222.7305473017085 241.06493423139753 L226.93697878993717 241.0278513100271 L231.14341027816585 240.77961786863844 L235.34984176639452 240.70746159036042 L239.5562732546232 240.780839622281 L243.76270474285187 240.80568871699873 L247.96913623108054 240.86070850898682 L252.1755677193092 240.8885121065922 L256.3819992075379 240.9125123710619 L260.5884306957666 240.82324231502423 L264.79486218399524 241.1483681166198 L269.00129367222394 241.45785577787657 L273.2077251604526 241.76274349693324 L277.4141566486813 242.15224639970785 L281.62058813691 242.41211354731954 L285.82701962513863 242.8001234784711 L290.0334511133673 243.13454014507303 L294.239882601596 243.5536597123711 L298.4463140898247 243.95352987419673 L302.6527455780533 244.4750003743899 L306.85917706628203 245.01554172906577 L311.0656085545107 245.54600985510592 L315.2720400427394 246.06958270105923 L319.478471530968 246.62325764819394 L323.6849030191967 247.2842579698816 L327.8913345074254 247.8959592495634 L332.0977659956541 248.58381919489784 L336.3041974838827 249.29742788610986 L340.5106289721114 249.99042926889362 L344.7170604603401 250.67032302418195 L348.92349194856877 251.41954539998363 L353.12992343679747 252.16811331979432 L357.3363549250261 252.9165670649028 L361.5427864132548 253.66538051105314 L365.74921790148346 254.41484450105492 L369.9556493897122 255.16512611514048 L374.16208087794087 255.91631440384094 L378.3685123661695 256.6684594192351 L382.57494385439816 257.42157423069676 L386.7813753426269 258.1756487633006 L390.98780683085556 258.9306564392321 L395.1942383190842 259.6865520157213 L399.40066980731285 260.4432778593571 L403.6071012955416 261.2007693283015 L407.81353278377026 261.9589568502778 L412.01996427199896 262.7177678707485 L416.2263957602276 263.4771162258311 L420.4328272484563 264.23690853202197 L424.639258736685 264.99705372797007 L428.84569022491365 265.7574558306045 L433.05212171314236 266.518025748

Also we can stack areas (for that we need `x` coordinates to match - use `trim = true`):

In [19]:
groupedRangesDF.plot {
    // use trim
    densityPlot(column<Double>("range"), trim = true) {
        // // adjust position of areas from diffrente groups
        position = Position.stack()
        alpha = 0.8
    }
}

Line_30.jupyter.kts (1:1 - 16) Unresolved reference: groupedRangesDF
Line_30.jupyter.kts (3:17 - 40) Type mismatch: inferred type is ColumnAccessor<Double> but Iterable<Number> was expected
Line_30.jupyter.kts (3:55 - 7:6) Type mismatch: inferred type is () -> Unit but BandWidth was expected
Line_30.jupyter.kts (5:9 - 17) Unresolved reference. None of the following candidates is applicable because of receiver type mismatch: 
public var LayerContextInterface.position: Position defined in org.jetbrains.kotlinx.kandy.letsplot.position
Line_30.jupyter.kts (6:9 - 14) Unresolved reference: alpha

`densityPlot` plot for `GroupBy` (i.e. `GroupBy.densityPlot(statDensityArgs)` extensions) works as well:

In [20]:
groupedRangesDF.densityPlot("range", bandWidth = BandWidth.value(10.0))

Line_31.jupyter.kts (1:1 - 16) Unresolved reference: groupedRangesDF

... and can be configured the same way:

In [21]:
groupedRangesDF.densityPlot(n = 750, trim = true, adjust = 0.75) { x(range) }.configure {
    alpha = 0.6
    position = Position.stack()
    // can access key column by name as `String`
    fillColor("category") { scale = categoricalColorBrewer(BrewerPalette.Qualitative.Dark2) }
}

Line_32.jupyter.kts (1:1 - 16) Unresolved reference: groupedRangesDF
Line_32.jupyter.kts (1:68 - 69) Unresolved reference. None of the following candidates is applicable because of receiver type mismatch: 
public fun PlotContext.x(column: String, parameters: LetsPlotPositionalMappingParametersContinuous<Any?>.() -> Unit = ...): PositionalMapping<Any?> defined in org.jetbrains.kotlinx.kandy.letsplot
public fun <T> PlotContext.x(values: Iterable<TypeVariable(T)>, name: String? = ..., parameters: LetsPlotPositionalMappingParametersContinuous<TypeVariable(T)>.() -> Unit = ...): PositionalMapping<TypeVariable(T)> defined in org.jetbrains.kotlinx.kandy.letsplot
public fun <T> PlotContext.x(values: DataColumn<TypeVariable(T)>, parameters: LetsPlotPositionalMappingParametersContinuous<TypeVariable(T)>.() -> Unit = ...): PositionalMapping<TypeVariable(T)> defined in org.jetbrains.kotlinx.kandy.letsplot
public fun <T> PlotContext.x(column: ColumnReference<TypeVariable(T)>, parameters: LetsPlotPo

### Inside `groupBy{}` plot context

We can apply `groupBy` modification to the initial dataset and build a histogram with grouped data the same way:

In [22]:
rangesDF.plot {
    groupBy(category) {
        densityPlot(range)
    }
}

Line_33.jupyter.kts (1:1 - 9) Unresolved reference: rangesDF
Line_33.jupyter.kts (2:5 - 12) Unresolved reference. None of the following candidates is applicable because of receiver type mismatch: 
public fun <T> DataFrame<TypeVariable(T)>.groupBy(vararg cols: String): GroupBy<TypeVariable(T), TypeVariable(T)> defined in org.jetbrains.kotlinx.dataframe.api
public fun <T> DataFrame<TypeVariable(T)>.groupBy(vararg cols: KProperty<*>): GroupBy<TypeVariable(T), TypeVariable(T)> defined in org.jetbrains.kotlinx.dataframe.api
public fun <T> DataFrame<TypeVariable(T)>.groupBy(vararg cols: AnyColumnReference /* = ColumnReference<*> */, moveToTop: Boolean = ...): GroupBy<TypeVariable(T), TypeVariable(T)> defined in org.jetbrains.kotlinx.dataframe.api
public fun <T> DataFrame<TypeVariable(T)>.groupBy(moveToTop: Boolean = ..., cols: ColumnsSelector<TypeVariable(T), *> /* = ColumnsSelectionDsl<TypeVariable(T)>.(it: ColumnsSelectionDsl<TypeVariable(T)>) -> ColumnSet<*> */): GroupBy<TypeVariable(T), 